### Loading your API keys using .env
#### Example .env:
```python
REDCAP_API_URL='https://redcap.ucsf.edu/api/'
REDCAP_API_KEY='YOUR_API_KEY'
```

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
api_url = os.environ['REDCAP_API_URL']
api_key = os.environ['REDCAP_API_KEY']

### Import the REDCap project data

In [ ]:
from redcap import Project # This is actually PyCap, which is annoyingly named.

# Import the REDCap project using your API Key
redcap_project = Project(api_url, api_key)
# Get the data from all the records in the project
redcap_data = redcap_project.export_records()

In [ ]:
# Convert the output to a pandas DataFrame
import pandas as pd

df_redcap = pd.DataFrame(redcap_data)
df_redcap.head()

display(df_redcap.head())

# NOTE: You can also just export as a pandas DataFrame directly like this:
# redcap_data = redcap_project.export_records(format='df')

### Example 1:
- Dr. X wants to know how many patients in the database have a diagnosis of Alzheimer's Disease (AD).
    - Of those cases, how many were under/over 70 years old at testing?

In [ ]:
# The variable/column name we are looking for first is "diagnosis" and the value for Alzheimer's Disease is 1
# Now, let's filter the dataframe to only cases with Alzheimer's Disease
df_ad = df_redcap[df_redcap['diagnosis'] == "1"]

# Print the total number of AD cases:
print(f"Total number of AD cases: {len(df_ad)}")

In [ ]:
# Now let's split the group into two groups based on the age cut off of 70 in the variable "visit_age"
age_cutoff = "70"
df_ad_young = df_ad[df_ad['visit_age'] < age_cutoff]
df_ad_old = df_ad[df_ad['visit_age'] >= age_cutoff]

# Print how many cases are in each group
print(f"Number of Alzheimer's Disease cases under {age_cutoff} years old: {len(df_ad_young)}")
print(f"Number of Alzheimer's Disease cases over {age_cutoff} years old: {len(df_ad_old)}")

### Example 2:
- Dr. X now wants to know the amounts of all diagnoses but displayed in a nice, labeled pie chart.

In [ ]:
# Let's take the original data and create a pie chart of the distribution of the different diagnoses
import matplotlib.pyplot as plt

# Plot the distribution of the different diagnoses
plt.figure(figsize=(4, 4), dpi=100)
df_redcap['diagnosis'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Diagnoses')
plt.show()

In [ ]:
# That's nice, but the number values aren't very informative and the NaN values are not labeled.
# Let's try replacing the values with labels using the data dictionary / metadata.

# You can extract the metadata of the project using PyCap like this:
metadata = redcap_project.export_metadata(format_type='df')
# Reset the index of the header rows:
metadata = metadata.groupby('field_name').first().reset_index()
metadata

In [ ]:
# First, let's extract the choices for the "diagnosis" variable from the metadata df 
# by finding the 'field_name' key with the value 'diagnosis' and then returning the 'select_choices_or_calculations' key:
# Return the value for the 'select_choices_or_calculations' key
diagnosis_choices = metadata[metadata['field_name'] == 'diagnosis']['select_choices_or_calculations'].values[0]
# Make the choices into a dict:
diagnosis_choices_dict = dict(item.split(", ") for item in diagnosis_choices.split(" | "))
diagnosis_choices_dict


In [ ]:
# Great! Now that we have the relabel dict, we can use it to replace the values in the 'diagnosis' column
# with the actual labels in the plot. But! We still haven't handled the blank/NaN values.
# Let's add to the relable dict a key for the blank/NaN values and call them "Unknown"
diagnosis_choices_dict[''] = 'Unknown'
diagnosis_choices_dict

In [ ]:
# Now, let's plot the distribution of the diagnoses again but relabeled:
plt.figure(figsize=(4, 4), dpi=100)
df_redcap['diagnosis'].replace(diagnosis_choices_dict).value_counts().plot.pie(autopct='%1.1f%%')
plt.ylabel('') # Remove the "count" label
plt.title('Diagnoses')
plt.show()

### Example 3:
- Dr. X now wants this kind of pie chart for every other categorical variable too (e.g. Sex, Handedness, etc.)

In [ ]:
# Rather than doing this all manually every time, let's semi-automate this
# by creating a function to make pie charts for all categorical variables in the data.

def categorical_pie(df, metadata):
    # Find every categorical variable in the metadata (e.g. 'radio', 'dropdown')
    categorical_vars = metadata[metadata['field_type'].str.contains('dropdown|radio', case=False)]
    # Get the field_name, field_label, and selected choices for each categorical variable
    for index, row in categorical_vars.iterrows():
        field_name = row['field_name']
        field_label = row['field_label']
        choices = dict(item.split(", ") for item in row['select_choices_or_calculations'].split(" | "))
        # Add the "Unknown" label for blank/NaN values:
        choices[''] = 'Unknown'
        # Plot the pie chart
        plt.figure(figsize=(4, 4), dpi=100)
        df[field_name].replace(choices).value_counts().plot.pie(autopct='%1.1f%%')
        plt.ylabel('')
        plt.title(field_label)

categorical_pie(df_redcap, metadata)

### More complicated example:
- Dr. X has an important grant submission coming up and needs the following:
    - A table with Sex, Handedness, Age at Visit, and MMSE by Diagnosis
    - Also, plot the MMSE scores by diagnosis.

In [ ]:
# Here is a function that generates a table of demographics for a given group variable
# with categorical and continuous variable lists, replaces the variable names and values with labels,
# adds percentages and counts for categorical variables, and means and standard deviations for continuous variables.

import numpy as np

def create_demographics_table(df, metadata, group_var, categorical_vars, continuous_vars):
    """
    Create a demographics table displaying mean ± SD (n=...) for continuous variables and
    percentages (n=...) for categorical variables, grouped by a specified variable.
    Variable names and codes are replaced with their labels from the metadata.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame containing the dataset.
    metadata (pd.DataFrame): The REDCap metadata DataFrame.
    group_var (str): The name of the grouping variable.
    categorical_vars (list): A list of names of the categorical variables.
    continuous_vars (list): A list of names of the continuous variables.
    
    Returns:
    pd.DataFrame: A formatted demographics table.
    """
    # Create a deep copy of the DataFrame to prevent in-place modifications
    df = df.copy(deep=True)
    
    # Create a mapping from field_name to field_label
    field_label_mapping = dict(zip(metadata['field_name'], metadata['field_label']))
    
    # Replace variable names with their labels
    group_var_label = field_label_mapping.get(group_var, group_var)
    continuous_vars_labels = [field_label_mapping.get(var, var) for var in continuous_vars]
    categorical_vars_labels = [field_label_mapping.get(var, var) for var in categorical_vars]    
    
    # Replace codes with labels in the grouping variable if it's categorical
    group_var_field_type = metadata.loc[metadata['field_name'] == group_var, 'field_type'].values
    if len(group_var_field_type) > 0 and group_var_field_type[0] in ['dropdown', 'radio']:
        choices_str = metadata.loc[metadata['field_name'] == group_var, 'select_choices_or_calculations'].values[0]
        choices_dict = dict(item.strip().split(', ', 1) for item in choices_str.split(' | '))
        # Include "" as a category labeled "Unknown"
        choices_dict[''] = 'Unknown'
        df[group_var] = df[group_var].astype(str).replace({'nan': ''}).map(choices_dict).fillna('Unknown')
    else:
        # For non-dropdown/radio fields, treat NaN as "Unknown"
        df[group_var] = df[group_var].astype(str).replace({'nan': ''}).fillna('Unknown')
    
    # Process categorical variables to replace codes with labels
    for var in categorical_vars:
        field_type = metadata.loc[metadata['field_name'] == var, 'field_type'].values
        if len(field_type) > 0 and field_type[0] in ['dropdown', 'radio']:
            choices_str = metadata.loc[metadata['field_name'] == var, 'select_choices_or_calculations'].values[0]
            choices_dict = dict(item.strip().split(', ', 1) for item in choices_str.split(' | '))
            # Include "" as a category labeled "Unknown"
            choices_dict[''] = 'Unknown'
            df[var] = df[var].astype(str).replace({'nan': ''}).map(choices_dict).fillna('Unknown')
        else:
            # For non-dropdown/radio fields, treat NaN as "Unknown"
            df[var] = df[var].astype(str).replace({'nan': ''}).fillna('Unknown')
    
    # Ensure continuous variables are numeric
    for var in continuous_vars:
        df[var] = pd.to_numeric(df[var], errors='coerce')
    
    # Initialize an empty list to collect results
    results = []
    
    # Get unique groups, including 'Unknown' for missing values
    groups = df[group_var].unique()
    
    # Process continuous variables
    for i, var in enumerate(continuous_vars):
        var_label = continuous_vars_labels[i]
        # Group by the group variable and calculate mean, standard deviation, and count
        grouped = df.groupby(group_var)[var].agg(['mean', 'std', 'count'])
        for grp in groups:
            mean = grouped.loc[grp, 'mean']
            std = grouped.loc[grp, 'std']
            count = int(grouped.loc[grp, 'count'])
            if pd.isnull(mean) or pd.isnull(std):
                value = "NaN"
            else:
                value = f"{mean:.2f} ± {std:.2f} (n={count})"
            results.append({
                'Variable': var_label,
                'Group': grp,
                'Value': value
            })
    
    # Process categorical variables
    for i, var in enumerate(categorical_vars):
        var_label = categorical_vars_labels[i]
        categories = df[var].dropna().unique()
        # Ensure "Unknown" is included in categories if any missing values
        if 'Unknown' in df[var].values and 'Unknown' not in categories:
            categories = np.append(categories, 'Unknown')
        for category in categories:
            for grp in groups:
                grp_data = df[df[group_var] == grp]
                total_count = len(grp_data)
                if total_count == 0:
                    pct = 0
                    n = 0
                else:
                    count = grp_data[var].value_counts().get(category, 0)
                    pct = (count / total_count) * 100
                    n = count
                # Replace empty category labels with "Unknown"
                category_label = category if category != '' else 'Unknown'
                results.append({
                    'Variable': f"{var_label} ({category_label})",
                    'Group': grp,
                    'Value': f"{pct:.1f}% (n={n})"
                })
    
    # Convert results to DataFrame
    result_df = pd.DataFrame(results)
    
    # Pivot the table to have groups as columns
    table = result_df.pivot(index='Variable', columns='Group', values='Value')
    
    # Optional: Reorder the index to display continuous variables first
    variable_order = continuous_vars_labels + [
        f"{field_label_mapping.get(var, var)} ({cat if cat != '' else 'Unknown'})"
        for var, var_label in zip(categorical_vars, categorical_vars_labels)
        for cat in df[var].dropna().unique()
    ]
    table = table.reindex(variable_order)
    
    # Reset index for better display
    table = table.reset_index()
    
    # Rename the 'Group' columns to include the group variable label
    table.columns.name = group_var_label
    
    return table


In [ ]:
# Specify variables/columns:
group_var = 'diagnosis'
categorical_vars = ['sex', 'handedness']
continuous_vars = ['visit_age', 'mmse_tot']

# Create the demographics table
demographics_table = create_demographics_table(df_redcap, metadata, group_var, categorical_vars, continuous_vars)
display(demographics_table)


In [ ]:
# Now to make the plot, we will go ahead and make a reusable plotting function also
# so that we can use it for other comparisons in the future as well.

import seaborn as sns

def plot_continuous_variable_by_group(df, metadata, continuous_var, group_var, bins=None):
    """
    Plots a histogram of a continuous variable grouped by a categorical variable with side-by-side bars.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame containing the dataset.
    metadata (pd.DataFrame): The REDCap metadata DataFrame.
    continuous_var (str): The name of the continuous variable in the DataFrame.
    group_var (str): The name of the grouping variable in the DataFrame.
    bins (int or sequence): Number of bins or bin edges for the histogram.
    
    Returns:
    None: Displays the histogram plot.
    """
    # Create a copy of the DataFrame to prevent in-place modifications
    df = df.copy(deep=True)
    
    # Map codes to labels for the grouping variable using metadata
    # Get the field_label for the grouping variable
    group_var_label = metadata.loc[metadata['field_name'] == group_var, 'field_label'].values
    if len(group_var_label) > 0:
        group_var_label = group_var_label[0]
    else:
        group_var_label = group_var
    
    # Map codes to labels for the grouping variable
    group_var_field_type = metadata.loc[metadata['field_name'] == group_var, 'field_type'].values
    if len(group_var_field_type) > 0 and group_var_field_type[0] in ['dropdown', 'radio']:
        choices_str = metadata.loc[metadata['field_name'] == group_var, 'select_choices_or_calculations'].values[0]
        choices_dict = dict(item.strip().split(', ', 1) for item in choices_str.split(' | '))
        # Include "" as a category labeled "Unknown"
        choices_dict[''] = 'Unknown'
        df[group_var] = df[group_var].astype(str).replace({'nan': ''}).map(choices_dict).fillna('Unknown')
    else:
        df[group_var] = df[group_var].astype(str).replace({'nan': ''}).fillna('Unknown')
    
    # Map codes to labels for the continuous variable if necessary
    continuous_var_label = metadata.loc[metadata['field_name'] == continuous_var, 'field_label'].values
    if len(continuous_var_label) > 0:
        continuous_var_label = continuous_var_label[0]
    else:
        continuous_var_label = continuous_var
    
    # Ensure continuous variable is numeric
    df[continuous_var] = pd.to_numeric(df[continuous_var], errors='coerce')
    
    # Drop rows with NaN in continuous variable
    df = df.dropna(subset=[continuous_var])
    
    # Define bins if not provided
    if bins is None:
        bins = range(int(df[continuous_var].min()), int(df[continuous_var].max()) + 2)  # +2 to include max value
    
    # Create a pivot table to get counts per bin per group
    df['bin'] = pd.cut(df[continuous_var], bins=bins, right=False, include_lowest=True)
    pivot_table = df.pivot_table(index='bin', columns=group_var, values=continuous_var, aggfunc='count', fill_value=0, observed=False)
    
    # Prepare data for plotting
    bin_labels = [f"{int(interval.left)}" for interval in pivot_table.index]
    groups = pivot_table.columns.tolist()
    counts = pivot_table.values
    
    # Plotting
    x = np.arange(len(bin_labels))  # the label locations
    total_width = 0.8
    num_groups = len(groups)
    width = total_width / num_groups  # the width of the bars
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Generate color palette
    colors = sns.color_palette('Set2', num_groups)
    
    # Plot each group's bars
    for i, group in enumerate(groups):
        ax.bar(x + i * width - total_width / 2, counts[:, i], width, label=str(group), color=colors[i])
    
    # Set labels and titles
    ax.set_xlabel(continuous_var_label)
    ax.set_ylabel('Frequency')
    ax.set_title(f'Distribution of {continuous_var_label} by {group_var_label}')
    ax.set_xticks(x)
    ax.set_xticklabels(bin_labels)
    ax.legend(title=group_var_label)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the MMSE scores by Diagnoses
plot_continuous_variable_by_group(df_redcap, metadata, continuous_var='mmse_tot', group_var='diagnosis')

In [ ]:
# Clearly the controls have higher MMSE scores (by definition) than the dementia cases, 
# so let's rerun the graph on a subset without the controls or "unknown" diagnoses for clarity:
df_mmse_trim = df_redcap[(df_redcap['diagnosis'] != '5') & (df_redcap['diagnosis'] != '')]

# Plot the MMSE scores by Diagnoses
plot_continuous_variable_by_group(df_mmse_trim, metadata, continuous_var='mmse_tot', group_var='diagnosis')

In [ ]:
# Since we made a reusable function, we can also use it to plot other continuous variables by group.
# Like Age vs Sex (with the unknowns removed):
df_sex_trim = df_redcap[(df_redcap['sex'] != '')]

plot_continuous_variable_by_group(df_sex_trim, metadata, 'visit_age', 'sex', bins=15)